In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import konlpy
from konlpy.tag import Twitter
from collections import Counter
import pickle as pickle
from tqdm import tnrange
from tqdm import tqdm

## 01. 기업별 빈도사전

In [2]:
essay_df = pd.read_pickle('essay_df_2.pkl')

In [4]:
essay_df.head()

,company_id,company,year,season,experience,position,score,feedback,content,views,position_broad
0,0,농협은행(주),2020,상반기,신입,사무·총무·법무,2.0,전반적으로 호감을 주지 못하는 내용 성실히 쓰려고 노력한 점 질문의 의도에 맞지 ...,직장은 생계를 꾸려나가는 수단이기도 하지만 저의 가치관을 실현시킬 수 있는 곳이라고...,"15,387",경영·사무
1,0,농협은행(주),2019,하반기,신입,사무·총무·법무,3.0,"지원회사 및 관련 산업, 직무에 대한 직접적인 준비과정이 없음 질문에 적절한 이...","2015년, ㅇㅇㅇㅇㅇㅇㅇ 소속 사업단으로 'ㅇㅇ ㅇㅇㅇ 식품 박람회'에 참가해, ...","5,470",경영·사무
2,0,농협은행(주),2019,하반기,신입,사무·총무·법무,2.0,회사 및 직무와 관련하여 무엇을 준비하고 노력했는지 알 수 없음 문장이 잘 정리...,글로벌 감각을 키우고 농업에 대한 이해도를 높이기 위한 글로벌 OOO 캠프의 외국인...,"1,591",경영·사무
3,0,농협은행(주),2019,하반기,신입,사무·총무·법무,2.0,지원자가 이 회사에서 무엇을 하고 싶은 지 잘 모르겠음 문장이 잘 정리되어 있음...,"디지털, 산업화로 인한 폐해가 커질수록, 생명에 영향을 끼치는 농업의 중요성이 커지...","2,169",경영·사무
4,0,농협은행(주),2019,하반기,신입,사무·총무·법무,3.0,실제 업무 경험을 사례로 제시한 점은 좋으나 회사 및 직무와 직접적인 연관이 없고...,"자동차 제조사의 금융 제휴 유치를 위해, 협력과 동기부여를 바탕으로 전속 금융사로 ...","4,954",경영·사무


In [3]:
#한국어 불용어
stopwords = pd.read_json('stopwords-ko.json')
stopwords_list = list(stopwords[0])
stopwords_list

['!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '...',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '\\',
 '^',
 '_',
 '`',
 '|',
 '~',
 '·',
 '—',
 '——',
 '‘',
 '’',
 '“',
 '”',
 '…',
 '、',
 '。',
 '〈',
 '〉',
 '《',
 '》',
 '가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도

In [4]:
#contents(자소서 내용) 을 넣었을 때 빈도분

common_del_list = ['되어다','때문','그리고','좋다','자다','없다','같다','싶다','보다', '하다', '있다', '되다', '이다', '돼다', '않다', '그렇다', '아니다', '이렇다', '그렇다', '어떻다']
del_list = stopwords_list + common_del_list
counter_list = []

def counter(contents): 
    
    twitter = Twitter()
    raw_pos_tagged = twitter.pos(contents, norm=True, stem=True) 
 
    word_cleaned = []
    
    for word, tag in raw_pos_tagged: 
        if not tag in ["Josa", "Eomi", "Punctuation", "Foreign"]: 
            if (len(word) != 1) & (word not in del_list): 
                word_cleaned.append(word)

    word_counted = Counter(word_cleaned)
    word_dic = dict(word_counted)
    
    counter_list.append(word_dic)
    return word_dic


In [15]:
content_list = []

for x in range (11):
    contents =''
    for i in tnrange(len(essay_df.content[essay_df['company_id']==x])):
        contents += essay_df.content[essay_df['company_id']==x].values[i]
    content_list.append(contents)
    counter(contents)

/Users/haeunkang/anaconda3/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [24]:
sorted_counter_list = []

for di in counter_list:
    sorted_counter = sorted(di.items(), key=lambda x:x[1], reverse=True)
    sorted_counter_list.append(sorted_counter)

In [26]:
with open('content_list.pkl', 'wb') as f:
    pickle.dump(content_list, f)

with open('counter_list.pkl', 'wb') as f:
    pickle.dump(counter_list, f)
    
with open('sorted_counter_list.pkl', 'wb') as f:
    pickle.dump(sorted_counter_list, f)

In [17]:
with open('company_code_dict.pkl', 'rb') as f:
    company_code_dict = pickle.load(f)

In [30]:
for i in range(11):
    print(company_code_dict[i], '의 빈도사전입니다:', '\n')
    print(sorted_counter_list[i][:50], '\n')

농협 의 빈도사전입니다: 

[('농협', 553), ('생각', 466), ('고객', 448), ('통해', 330), ('위해', 287), ('은행', 231), ('경험', 226), ('업무', 210), ('노력', 187), ('많다', 183), ('목표', 170), ('금융', 166), ('받다', 165), ('대한', 150), ('사람', 148), ('이르다', 140), ('만들다', 139), ('농업인', 135), ('발전', 131), ('사회', 129), ('농업', 124), ('서비스', 123), ('상품', 123), ('가장', 119), ('IT', 117), ('활동', 116), ('NH', 116), ('지역', 113), ('이러하다', 113), ('필요하다', 112), ('농민', 111), ('상황', 109), ('새롭다', 108), ('문제', 106), ('결과', 106), ('사업', 104), ('바탕', 104), ('진행', 103), ('중요하다', 99), ('크다', 99), ('다양하다', 98), ('지원', 97), ('소통', 97), ('가지', 96), ('얻다', 93), ('신뢰', 92), ('제공', 92), ('가치', 88), ('오다', 88), ('분석', 88)] 

롯데 의 빈도사전입니다: 

[('고객', 716), ('생각', 605), ('경험', 586), ('업무', 562), ('통해', 546), ('롯데', 533), ('위해', 504), ('영업', 419), ('사람', 384), ('관리', 373), ('노력', 321), ('배우다', 313), ('대한', 312), ('많다', 300), ('만들다', 269), ('받다', 255), ('활동', 249), ('가지', 240), ('이르다', 240), ('다양하다', 232), ('목표', 232), ('결과', 228), ('분석', 223), ('입사', 20

## 02. 기업별 빈도사전 (불용어 처리 심화 후)

In [6]:
del_words = ['생각', '통해', '위해', '업무','경험', '대한', '노력', '회사', '배우다', '결과', '다양하다',
             '직무', '크다','가장', '받다', '분야', '과정', '이르다', '다양하다', '수행', '전공', '맡다',
             '모비스', '이루다', '영업', '시작', '오다', '필요하다', '새롭다', '현대', '인턴', '현장',
             '나가다', '관련', '시장', '해보다', '산업', '역할', '발전', '가다', '사업', '모비스', '부서',
             '직접', '입사', '상황', '영업', '동아리', '갖추다', '사용', '통해', '위해', '만들다', '받다',
             '이르다', '이러하다', '얻다', '가다', '대해', '중요하다', '필요하다', '대한', '배우다', '많다',
             '다양하다', '새롭다', '들다', '맡다', '오다', '늘다', '이루다', '통해', '위해', '많다',
             '이러하다', '필요하다', '얻다', '오다', '크다', '가지', '대한', '이르다', '느끼다', '많다',
             '해주다', '모든', '내다', '맡다', '들다', '나가다', '가다', '가지다', '받다', '만들다',
             '배우다', '해주다', '따르다', '이루다', '주다', '내다', '여러', '생각', '받다', '만들다',
             '배우다', '키우다', '많다', '통해', '위해', '대한', '이르다', '새롭다', '크다', '이러하다',
             '느끼다', '들다', '필요하다', '얻다', '중요하다', '가다', '맡다', '이루다', '이러하다', '다양하다',
             '필요하다', '오다', '해보다', '가장', '해주다', '가다' , '크다', '전기','이르다', '가지',
             '이러하다', '대해', '필요하다', '가장', '크다', '바탕', '오다', '들다', 
             '농협', '삼성', 'GS', 'KT', '롯데', '한국전력공사', '한화', 'CJ', 'LG', 'SK']

del_words_set = set(del_words)

In [7]:
common_del_list = ['되어다','때문','그리고','좋다','자다','없다','같다','싶다','보다', '하다', '있다', '되다', '이다', '돼다', '않다', '그렇다', '아니다', '이렇다', '그렇다', '어떻다']
del_list = stopwords_list + common_del_list + list(del_words_set)
counter_list_2 = []

def counter(contents): 
    
    twitter = Twitter()
    raw_pos_tagged = twitter.pos(contents, norm=True, stem=True) 
 
    word_cleaned = []
    
    for word, tag in raw_pos_tagged: 
        if not tag in ["Josa", "Eomi", "Punctuation", "Foreign"]: 
            if (len(word) != 1) & (word not in del_list): 
                word_cleaned.append(word)

    word_counted = Counter(word_cleaned)
    word_dic = dict(word_counted)
    
    counter_list_2.append(word_dic)
    return word_dic


In [9]:
content_list_2 = []

for x in range (11):
    contents =''
    for i in range(len(essay_df.content[essay_df['company_id']==x])):
        contents += essay_df.content[essay_df['company_id']==x].values[i]
    content_list_2.append(contents)
    counter(contents)

In [39]:
sorted_counter_list_2 = []

for di in counter_list_2:
    sorted_counter = sorted(di.items(), key=lambda x:x[1], reverse=True)
    sorted_counter_list_2.append(sorted_counter)

In [10]:
with open('content_list_2.pkl','wb') as f:
    pickle.dump(content_list_2, f)

with open('counter_list_2.pkl','wb') as f:
    pickle.dump(counter_list_2, f)
    
with open('sorted_counter_list_2.pkl', 'wb') as f:
    pickle.dump(sorted_counter_list_2, f)

In [40]:
for i in range(11):
    print(company_code_dict[i], '의 빈도사전입니다:', '\n')
    print(sorted_counter_list_2[i][:50], '\n')

농협 의 빈도사전입니다: 

[('고객', 448), ('은행', 231), ('목표', 170), ('금융', 166), ('사람', 148), ('농업인', 135), ('사회', 129), ('농업', 124), ('상품', 123), ('서비스', 123), ('IT', 117), ('활동', 116), ('NH', 116), ('지역', 113), ('농민', 111), ('문제', 106), ('진행', 103), ('지원', 97), ('소통', 97), ('제공', 92), ('신뢰', 92), ('가치', 88), ('분석', 88), ('시스템', 88), ('직원', 85), ('역량', 85), ('도전', 85), ('참여', 84), ('교육', 84), ('경제', 79), ('학생', 77), ('친구', 77), ('기술', 76), ('기업', 76), ('달성', 74), ('먼저', 72), ('해결', 72), ('관리', 72), ('이해', 71), ('능력', 71), ('프로젝트', 70), ('항상', 67), ('행사', 66), ('기획', 66), ('근무', 66), ('생활', 65), ('모습', 65), ('파악', 64), ('책임감', 64), ('의견', 64)] 

롯데 의 빈도사전입니다: 

[('고객', 716), ('사람', 384), ('관리', 373), ('활동', 249), ('목표', 232), ('분석', 223), ('상품', 202), ('제품', 201), ('매장', 200), ('능력', 190), ('공부', 179), ('서비스', 178), ('역량', 174), ('진행', 174), ('도전', 174), ('지원', 172), ('매출', 170), ('판매', 169), ('소통', 169), ('기업', 162), ('파악', 158), ('문제', 155), ('항상', 152), ('전략', 150), ('지식', 140), ('해결', 136), ('

## 03. 직무별 빈도사전

In [14]:
with open('position_wide.pkl', 'rb') as f:
    position_wide = pickle.load(f)

In [33]:
position_counter_list = []

def position_counter(contents): 
    
    twitter = Twitter()
    raw_pos_tagged = twitter.pos(contents, norm=True, stem=True) 
 
    word_cleaned = []
    
    for word, tag in raw_pos_tagged: 
        if not tag in ["Josa", "Eomi", "Punctuation", "Foreign"]: 
            if (len(word) != 1) & (word not in del_list): 
                word_cleaned.append(word)

    word_counted = Counter(word_cleaned)
    word_dic = dict(word_counted)
    
    position_counter_list.append(word_dic)
    return word_dic

In [34]:
for posit in position_wide:
    contents =''
    for i in range(len(essay_df.content[essay_df['position_broad'] == posit])):
        contents += essay_df.content[essay_df['position_broad'] == posit].values[i]
    position_counter(contents)
    
with open('position_counter_list.pkl', 'wb') as f:
    pickle.dump(position_counter_list, f)

In [16]:
sorted_position = []

for di in position_counter_list:
    sorted_counter = sorted(di.items(), key=lambda x:x[1], reverse=True)
    sorted_position.append(sorted_counter)

In [45]:
for i in range(14):
    print(position_wide[i], '의 빈도사전입니다:', '\n')
    print(sorted_position[i][:50], '\n')

경영·사무 의 빈도사전입니다: 

[('고객', 649), ('관리', 582), ('사람', 573), ('활동', 555), ('기업', 515), ('목표', 513), ('분석', 451), ('능력', 428), ('역량', 398), ('회계', 380), ('도전', 373), ('진행', 368), ('기획', 365), ('소통', 360), ('지식', 353), ('재무', 350), ('공부', 345), ('지원', 339), ('이해', 320), ('문제', 313), ('해결', 305), ('학생', 305), ('인재', 302), ('조직', 300), ('인사', 296), ('활용', 285), ('준비', 271), ('효율', 264), ('직원', 260), ('운영', 250), ('교육', 249), ('계획', 249), ('생활', 247), ('문화', 244), ('봉사', 240), ('프로젝트', 235), ('수업', 230), ('참여', 229), ('사회', 226), ('가치', 226), ('팀원', 222), ('방법', 219), ('파악', 218), ('작성', 217), ('자동차', 217), ('전략', 216), ('의견', 216), ('서비스', 213), ('근무', 210), ('찾다', 209)] 

마케팅·광고·홍보 의 빈도사전입니다: 

[('마케팅', 377), ('고객', 349), ('분석', 252), ('기획', 219), ('목표', 184), ('사람', 182), ('활동', 162), ('진행', 158), ('전략', 135), ('광고', 126), ('제품', 122), ('기업', 121), ('데이터', 113), ('홍보', 111), ('능력', 109), ('도전', 109), ('콘텐츠', 108), ('역량', 106), ('문제', 104), ('활용', 101), ('가치', 100), ('소비자', 98), ('프로젝트', 98

In [49]:
position_code_dic = {}

for i in range(len(position_wide)):
    position_code_dic[i] = position_wide[i]
    
with open('position_code_dic.pkl', 'wb') as f:
    pickle.dump(position_code_dic, f)

In [18]:
with open('sorted_position.pkl', 'wb') as f:
    pickle.dump(sorted_position, f)